In [1]:
import warnings; warnings.simplefilter('ignore')    
import pandas as pd
import numpy as np
# import statsmodels.api as sm
# import scipy.stats as scs
# import tushare as ts
%matplotlib inline
# from scipy.stats import norm
from matplotlib import pyplot as plt
# from scipy.interpolate import PchipInterpolator
# from scipy.integrate import quad
# from scipy import stats
from tqdm import *

## import and process liquidity.csv
### feature: ['Exit price']

In [2]:
liquidity = pd.read_csv('investmentsUK\liquidity.csv')
liquidity = liquidity.drop(columns=['Beauhurst URL','Companies House Number'])    #drop useless information
liquidity = liquidity.rename(columns={"Date of the company's exit or death":"Exit date","Current Stage of Evolution":"Current stage"})
liquidity = liquidity.sort_values(by=['Company name']) #sort the data by company name

In [3]:
liquidity.head()

,Company name,Incorporation date,Current stage,Exit date,Exit price
641,10 Digital Media,NaN,Dead,27/11/2018,0.0
1020,10 Minutes With,19/03/2013,Dead,29/06/2017,0.0
3197,1066 Airsoft,13/05/2014,Dead,27/02/2019,0.0
1076,14M Genomics,09/01/2014,Dead,21/03/2016,0.0
3358,1855,27/02/2013,Dead,20/02/2020,0.0


## import and process deals.csv
### feature: ['Investment amount']

In [4]:
deals = pd.read_csv('investmentsUK\deals.csv')    #import data
deals = deals.drop(columns=['Beauhurst company URL','Beauhurst deal URL','Amount raised (converted to GBP)'])    #drop useless information
deals = deals.rename(columns={'Verified investment amount (converted to GBP)':'Investment amount'})    #rename column
deals = deals.dropna()    #clean missing data rows
deals['Deal date'] = pd.to_datetime(deals['Deal date'],format='%Y-%m-%d')
deals = deals.sort_values(by=['Company name'])    #sort the data by company name

In [5]:
deals.head()

,Company name,Deal date,Investment amount
3866,10 Digital Media,2015-11-16,3398454.0
6303,1066 Airsoft,2016-05-31,23668.0
1395,14M Genomics,2014-12-16,5000000.0
6787,1855,2017-06-30,20000.0
8443,1st Impression,2013-11-22,75005.0


## left merge deals.csv and liquidity.csv to get startups.csv
### feature: ['Investment amount', 'Exit price']

In [6]:
startups = pd.merge(deals, liquidity, on='Company name',how='left')    #left join deals.csv with liquidity.csv which have been processed
startups.loc[(startups['Current stage'] == "Dead") ,'Exit price'] = 0    #put zero in the 'Exit price' column if the startup is dead
startups = startups.dropna(subset=['Exit price'])    #clean data rows whose values in "Exit price" are missing
startups['Exit date'] = pd.to_datetime(startups['Exit date'],format='%d/%m/%Y')
startups['Exit date'] = pd.to_datetime(startups['Exit date'],format='%Y-%m-%d')

In [7]:
startups.head()

,Company name,Deal date,Investment amount,Incorporation date,Current stage,Exit date,Exit price
0,10 Digital Media,2015-11-16,3398454.0,NaN,Dead,2018-11-27,0.0
1,1066 Airsoft,2016-05-31,23668.0,13/05/2014,Dead,2019-02-27,0.0
2,14M Genomics,2014-12-16,5000000.0,09/01/2014,Dead,2016-03-21,0.0
3,1855,2017-06-30,20000.0,27/02/2013,Dead,2020-02-20,0.0
4,1st Impression,2013-11-22,75005.0,26/08/2009,Dead,2019-12-16,0.0


## import and process organizations.csv
### feature: ['first_industry']

In [8]:
organizations = pd.read_csv('investmentsUK/organizations.csv')
organizations = organizations.loc[:,['name', 'category_groups_list']]
organizations = organizations.rename(columns={'name':'Company name'})
organizations = organizations.dropna(inplace=False)

In [9]:
organizations['first_industry'] = organizations['category_groups_list'].apply(lambda x:x.split(',')[0])
map_df = pd.DataFrame(organizations['first_industry'].unique()).reset_index().rename(columns={0:'first_industry'})
organizations = pd.merge(organizations,map_df,on='first_industry',how='outer')
organizations = organizations.loc[:,['Company name','first_industry']]
organizations = organizations.sort_values(by=['Company name']) 

In [10]:
organizations.head()

,Company name,first_industry
909994,!Bewust sociaal op web,Design
680304,!Creatice,Education
512458,!DOEVE*,Advertising
787112,!FEST,Food and Beverage
722339,!K7,Media and Entertainment


In [11]:
# from sklearn import preprocessing

# data = (organizations.category_groups_list.str.split('\s*,\s*', expand=True)
#    .stack()
#    .str.get_dummies()
#    .sum(level=0))

# enc = preprocessing.OneHotEncoder(sparse=False)
# data = enc.fit_transform( organizations['first_industry'].values.reshape(-1,1) )

# organizations['one_hot'] = list(data)

## import and process funding_rounds.csv to get founder_count.csv
### feature: ['investor_count']

In [12]:
founder_count = pd.read_csv('investmentsUK\\funding_rounds.csv')
founder_count = founder_count.loc[:,['org_name','investor_count']]
founder_count = founder_count.rename(columns={'org_name':'Company name'}) 
founder_count = founder_count.dropna(inplace=False)
founder_count = founder_count.drop_duplicates(subset=None, keep='first', inplace=False)
founder_count = founder_count.sort_values(by=['Company name']) 

In [13]:
founder_count.head()

,Company name,investor_count
116667,!hey software,2.0
33185,#Vine,2.0
341803,#Yolo ex,1.0
106036,#hexology,1.0
266857,#paid,1.0


## left merge startups.csv and founder_count.csv
### feature: ['Investment amount', 'Exit price', 'founder_count']

In [14]:
startups = pd.merge(startups, founder_count, on='Company name',how='left') 

In [15]:
startups.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6635 entries, 0 to 6634
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Company name        6635 non-null   object        
 1   Deal date           6635 non-null   datetime64[ns]
 2   Investment amount   6635 non-null   float64       
 3   Incorporation date  6465 non-null   object        
 4   Current stage       6635 non-null   object        
 5   Exit date           6635 non-null   datetime64[ns]
 6   Exit price          6635 non-null   float64       
 7   investor_count      2735 non-null   float64       
dtypes: datetime64[ns](2), float64(3), object(3)
memory usage: 466.5+ KB


## import and process jobs.csv & people.csv to get founder_gender.csv
### feature: ['male', 'female']

In [16]:
jobs = pd.read_csv('investmentsUK/jobs.csv') 
jobs = jobs.loc[:,['person_name', 'org_name', 'title']]
jobs = jobs.rename(columns={'person_name':'founder_name'})
jobs = jobs.rename(columns={'org_name':'Company name'})
jobs = jobs.dropna(inplace=False)
searchfor = ['found','Found']
jobs = jobs[jobs['title'].str.contains('|'.join(searchfor))] 
#search for 'founder'/'co-founder'/'founding partner'
Cname = startups[['Company name']]
jobs = pd.merge(Cname,jobs,on='Company name',how='left')
jobs = jobs.dropna(subset=['founder_name','title'],inplace=False)
jobs = jobs.sort_values(by=['Company name']) 

In [17]:
jobs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5266 entries, 12 to 9376
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Company name  5266 non-null   object
 1   founder_name  5266 non-null   object
 2   title         5266 non-null   object
dtypes: object(3)
memory usage: 164.6+ KB


In [18]:
people = pd.read_csv('investmentsUK/people.csv') 
people = people.loc[:,['name', 'gender']]
people = people.rename(columns={'name':'founder_name'})
founder = pd.merge(jobs,people,on='founder_name',how='left')
founder = founder.drop_duplicates(subset=None, keep='first', inplace=False) 

In [19]:
founder.head()

,Company name,founder_name,title,gender
0,3 Kinds of Ice,David Bailey,CEO & Co-Founder,male
19,3sun Group,Willie Dawson,Founder,male
20,4x4 Aviation,Thorsten Reinhardt,Founder & CEO,male
21,A Suit That Fits,Warren Bennett,Co-Founder,male
25,ACAL Energy,Andrew Creeth,Chief Technology Officer & Co-Founder,male


In [20]:
length_column = founder.shape[0]
founder['male'] = (0)
founder['female'] = (0)
founder['founder_count'] = (0)
for i in tqdm(range(0,length_column)):
    if (founder['Company name'].iloc[i] == founder['Company name'].iloc[i-1])&(founder['gender'].iloc[i] == 'male'):
        founder['male'].iloc[i] = founder['male'].iloc[i-1]+1
        founder['founder_count'].iloc[i] += 1
    elif (founder['Company name'].iloc[i] == founder['Company name'].iloc[i-1])&(founder['gender'].iloc[i] == 'female'):
        founder['female'].iloc[i] = founder['female'].iloc[i-1]+1
        founder['founder_count'].iloc[i] += 1
    elif founder['gender'].iloc[i] == 'male':
        founder['male'].iloc[i] = 1
    else:
        founder['female'].iloc[i] = 1
founder_gender = pd.concat([founder,founder.fillna(0)],axis=0)
founder_gender = founder_gender.drop_duplicates(subset=['Company name'], keep='last', inplace=False)
founder_gender = founder_gender.loc[:,['Company name','male','female','founder_count']]

100%|████████████████████████████████████████████████████████████████████████████| 1369/1369 [00:01<00:00, 1325.27it/s]


In [21]:
founder_gender

,Company name,male,female,founder_count
0,3 Kinds of Ice,1,0,0
19,3sun Group,1,0,0
20,4x4 Aviation,1,0,0
21,A Suit That Fits,1,0,0
25,ACAL Energy,1,0,0
...,...,...,...,...
9606,passingboxes,0,1,0
9608,people.io,1,0,0
9612,rtobjects,1,0,0
9617,uberVU,3,0,1


## left merge startups.csv and founder_count.csv
### feature: ['Investment amount', 'Exit price', 'founder_count', 'male', 'female']

In [22]:
startups = pd.merge(startups, founder_gender, on='Company name',how='left')
startups = startups.drop_duplicates(subset=['Company name','Deal date'], keep='first', inplace=False) 

In [23]:
startups.head()

,Company name,Deal date,Investment amount,Incorporation date,Current stage,Exit date,Exit price,investor_count,male,female,founder_count
0,10 Digital Media,2015-11-16,3398454.0,NaN,Dead,2018-11-27,0.0,NaN,NaN,NaN,NaN
1,1066 Airsoft,2016-05-31,23668.0,13/05/2014,Dead,2019-02-27,0.0,NaN,NaN,NaN,NaN
2,14M Genomics,2014-12-16,5000000.0,09/01/2014,Dead,2016-03-21,0.0,NaN,NaN,NaN,NaN
3,1855,2017-06-30,20000.0,27/02/2013,Dead,2020-02-20,0.0,NaN,NaN,NaN,NaN
4,1st Impression,2013-11-22,75005.0,26/08/2009,Dead,2019-12-16,0.0,NaN,NaN,NaN,NaN


## import and process shares.csv
### feature: ['number_of_shares', 'percent_total']

In [24]:
shares = pd.read_csv('investmentsUK\shares.csv')    #import data
shares = shares.loc[:,['Company name','filing_date', 'first_name', 'last_name', 'number_of_shares','percent_total']]    #select and retain useful information
shares = shares.dropna(subset=['last_name'])  
# shares = shares[~shares['number_of_shares'].isin([0])]    #clean data rows whose values in "number_of_shares" or "last_name" are missing
shares['first_name'] = shares['first_name'].fillna('')    #replace Nan to "" in "first_name" column
shares = shares.drop_duplicates(subset=None, keep='first', inplace=False)    #eliminate duplicate data rows
shares['filing_date'] = pd.to_datetime(shares['filing_date'],format='%d/%m/%Y')
shares['filing_date'] = pd.to_datetime(shares['filing_date'],format='%Y-%m-%d')
shares = shares.sort_values(by=['Company name','filing_date','last_name', 'number_of_shares'])    
#set Investor name as index and sort the data first by 'Company name' then by 'number_of_shares'
# investors = shares.drop_duplicates(subset=['Company name','number_of_shares'], keep='first', inplace=False)
#eliminate duplicate data rows whose number_of_shares make no change [179719 rows x 6 columns] --> [87229 rows x 6 columns]

In [25]:
shares.head()

,Company name,filing_date,first_name,last_name,number_of_shares,percent_total
33100,10 Digital Media,2015-06-18,,SPORTLOBSTER SA,100.0,100.0
33101,10 Digital Media,2016-06-18,,SPORTLOBSTER SA,100.0,100.0
33102,10 Digital Media,2017-06-18,,SPORTLOBSTER SA,100.0,100.0
33103,10 Digital Media,2018-06-18,,CONSTELLATION SPORTS LIMITED,100.0,100.0
52887,10 Minutes With,2013-03-19,MANFREDI,DI CINTIO,1000000.0,100.0


## Calculate how many shares have been issued each round 
### feature: ['round', 'round_shares']

In [26]:
rounds = shares.drop_duplicates(subset=['filing_date','Company name'], keep='last') 
rounds['total_shares'] = rounds['number_of_shares']*100/rounds['percent_total'] 
rounds = rounds.dropna(subset=['total_shares'])
#Calculate the cumulative number of shares issued by the startup firms in each funding round
# rounds.round({'total_shares': 2})
rounds = rounds.loc[:,['Company name','filing_date','total_shares']]
length_column = rounds.shape[0]
rounds['round_shares'] = ''
for i in tqdm(range(0,length_column)):
    if rounds['Company name'].iloc[i] == rounds['Company name'].iloc[i-1]:
        rounds['round_shares'].iloc[i] = rounds['total_shares'].iloc[i]-rounds['total_shares'].iloc[i-1]
    else:
        rounds['round_shares'].iloc[i] = rounds['total_shares'].iloc[i]
#Calculate the number of shares issued separately by the startup in each funding round

100%|██████████████████████████████████████████████████████████████████████████| 14982/14982 [00:08<00:00, 1821.71it/s]


In [27]:
rounds = rounds[~rounds['round_shares'].isin([0])] 
# if the value of round_shares is zero, there was not a new funding round
length_column = rounds.shape[0]
rounds['rounds'] = ''
for i in tqdm(range(0,length_column)):
    if rounds['Company name'].iloc[i] == rounds['Company name'].iloc[i-1]:
        rounds['rounds'].iloc[i] = rounds['rounds'].iloc[i-1]+1
    else:
        rounds['rounds'].iloc[i] = 1

100%|████████████████████████████████████████████████████████████████████████████| 8638/8638 [00:04<00:00, 1991.83it/s]


In [28]:
# company_name_lis = rounds['Company name'].unique().tolist()
# for cmp in tqdm(company_name_lis):
#     dd = rounds[rounds['Company name'] == cmp]
#     length_column = dd.shape[0]
#     dd['round_share'] = dd['number_of_shares'] - dd['number_of_shares'].shift()
#     dd['round'] = length_column
#     rounds = pd.concat([rounds,dd.fillna(0)],axis=0)

In [29]:
rounds

,Company name,filing_date,total_shares,round_shares,rounds
33100,10 Digital Media,2015-06-18,1.000000e+02,100.0,1
52887,10 Minutes With,2013-03-19,1.000000e+06,1000000.0,1
52891,10 Minutes With,2014-03-19,2.577320e+06,1577319.587629,2
52919,10 Minutes With,2015-03-19,3.048780e+06,471460.900176,3
52978,10 Minutes With,2016-03-19,3.244444e+06,195663.95664,4
...,...,...,...,...,...
136317,yWasteFood,2015-11-27,1.000000e+01,10.0,1
136318,yWasteFood,2016-11-26,1.399992e+01,3.999916,2
85764,âcasă,2017-07-19,1.628879e+06,1628878.894957,1
85804,âcasă,2018-07-19,2.411378e+06,782498.996066,2


## Select those start-ups who have raised fundings for 4 times

In [30]:
# startups = startups.drop_duplicates(subset=['Company name','Deal date'])
count_value=startups['Company name'].value_counts()
limit = list(count_value[count_value>3].index)
startups_limit = startups[startups['Company name'].isin(limit)]

In [31]:
count_value=rounds['Company name'].value_counts()
limit = list(count_value[count_value==4].index)
rounds_limit = rounds[rounds['Company name'].isin(limit)]
shares_limit = pd.merge(rounds_limit,shares, on=['Company name','filing_date'],how='left')

In [32]:
shares_limit['Company name'].value_counts()

The London Distillery Company    395
Wonga                            352
Ecco Recordings                  350
Neyber                           323
Metalysis                        310
                                ... 
IPS                                7
Oxitec                             4
CN Creative Group                  4
Tiller                             4
IOmet Pharma                       4
Name: Company name, Length: 325, dtype: int64

In [33]:
startups_limit['Company name'].value_counts()
# startups_limit.head()

Houseology              13
NetThings               12
Microtest Matrices      11
Streetlife              11
TenTel                   9
                        ..
Upstream                 4
Speed of Light Media     4
Klipfeed                 4
WatchScotch              4
Gazoob                   4
Name: Company name, Length: 321, dtype: int64

## Inner merge startups.csv and shares_limit.csv

In [34]:
df = pd.merge(shares_limit,startups_limit,on='Company name',how='inner')
# df = df[['Company name','Deal date','Investment amount','filing_date','Investor name','number_of_shares','percent_total','Current stage','Incorporation date','Exit date','Exit price']]
df = df.sort_values(by=['Company name','last_name','Deal date','filing_date']) 
df=df.loc[(df['Deal date']<df['filing_date'] )]
# drop deals whose deal date cannot match with filing date
df=df.drop_duplicates(subset=['filing_date','first_name','last_name','Company name'], keep='last') 
#match 'filing_date' with the closest 'Deal date'

In [35]:
# df['Company name'].value_counts()
df.info()
# df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8445 entries, 4 to 51433
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Company name        8445 non-null   object        
 1   filing_date         8445 non-null   datetime64[ns]
 2   total_shares        8445 non-null   float64       
 3   round_shares        8445 non-null   object        
 4   rounds              8445 non-null   object        
 5   first_name          8445 non-null   object        
 6   last_name           8445 non-null   object        
 7   number_of_shares    8445 non-null   float64       
 8   percent_total       8445 non-null   float64       
 9   Deal date           8445 non-null   datetime64[ns]
 10  Investment amount   8445 non-null   float64       
 11  Incorporation date  8241 non-null   object        
 12  Current stage       8445 non-null   object        
 13  Exit date           8445 non-null   datetime64[

## Calculate other features

In [36]:
exit_share = rounds.loc[:,['Company name','filing_date','total_shares']]
exit_share = exit_share.drop_duplicates(subset='Company name', keep='last')
exit_share = exit_share.rename(columns={'total_shares':'exit_shares'})
#Calculate the cumulative number of shares issued by the company at the time of investor exit
exit_share = exit_share.drop(columns='filing_date')
df = pd.merge(df, exit_share, on='Company name',how='left')

In [37]:
length_column = df.shape[0]
df['add_shares'] = ''
for i in tqdm(range(0,length_column)):
    if (df['last_name'].iloc[i] == df['last_name'].iloc[i-1])&(df['first_name'].iloc[i] == df['first_name'].iloc[i-1]):
        df['add_shares'].iloc[i] = df['number_of_shares'].iloc[i]-df['number_of_shares'].iloc[i-1]
    else:
        df['add_shares'].iloc[i] = df['number_of_shares'].iloc[i]

100%|████████████████████████████████████████████████████████████████████████████| 8445/8445 [00:06<00:00, 1357.72it/s]


In [38]:
df['follow'] = (0)
df.loc[(df['add_shares']>0) ,'follow'] = 1 #set the follow-on decisions as the label 

In [39]:
df['valuation'] = df['Investment amount']/df['round_shares']

In [40]:
df = pd.merge(df,organizations,on='Company name',how='left')

In [41]:
df

,Company name,filing_date,total_shares,round_shares,rounds,first_name,last_name,number_of_shares,percent_total,Deal date,...,Exit price,investor_count,male,female,founder_count,exit_shares,add_shares,follow,valuation,first_industry
0,ATEEDA,2013-06-30,2.739548e+05,273954.763537,1,JOHN COLIN,ADAMS,559.0,0.204,2012-05-24,...,0.0,NaN,NaN,NaN,NaN,4.297006e+05,559.0,1,3.650241,NaN
1,ATEEDA,2014-06-30,3.597177e+05,85762.934618,2,JOHN COLIN,ADAMS,1702.0,0.473,2014-05-31,...,0.0,NaN,NaN,NaN,NaN,4.297006e+05,1143.0,1,8.745013,NaN
2,ATEEDA,2015-06-30,3.469110e+05,-12806.70339,3,JOHN COLIN,ADAMS,1702.0,0.491,2014-05-31,...,0.0,NaN,NaN,NaN,NaN,4.297006e+05,0.0,0,-58.562924,NaN
3,ATEEDA,2016-06-30,4.297006e+05,82789.604038,4,JOHN COLIN,ADAMS,3082.0,0.717,2015-11-19,...,0.0,NaN,NaN,NaN,NaN,4.297006e+05,1380.0,1,7.247142,NaN
4,ATEEDA,2013-06-30,2.739548e+05,273954.763537,1,MICHAEL JAMES,ANDREW,720.0,0.263,2012-05-24,...,0.0,NaN,NaN,NaN,NaN,4.297006e+05,720.0,1,3.650241,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9391,tootle,2017-12-04,7.070543e+06,363924.9886,3,NINUO,ZHAO,12500.0,0.177,2017-09-27,...,0.0,NaN,NaN,NaN,NaN,8.217117e+06,0.0,0,3.649846,NaN
9392,tootle,2018-12-04,8.217117e+06,1146574.481458,4,NUNUO,ZHAO,12500.0,0.152,2018-06-25,...,0.0,NaN,NaN,NaN,NaN,8.217117e+06,12500.0,1,0.152556,NaN
9393,tootle,2017-11-24,6.706618e+06,3328239.26868,2,HEYKEL,ZOUITEN,9121.0,0.136,2017-09-27,...,0.0,NaN,NaN,NaN,NaN,8.217117e+06,9121.0,1,0.399091,NaN
9394,tootle,2017-12-04,7.070543e+06,363924.9886,3,HEYKEL,ZOUITEN,9121.0,0.129,2017-09-27,...,0.0,NaN,NaN,NaN,NaN,8.217117e+06,0.0,0,3.649846,NaN


### Calculate Step_up Valuation

In [42]:
step_up = df.loc[:,['Company name','filing_date','rounds','valuation']]
step_up = step_up.drop_duplicates(subset=['Company name','filing_date']).sort_values(by=['Company name','filing_date'])
length_column = step_up.shape[0]
step_up['value_chg'] = (0)
for i in tqdm(range(0,length_column)):
    if (step_up['Company name'].iloc[i] == step_up['Company name'].iloc[i-1]):
        step_up['value_chg'].iloc[i] = step_up['valuation'].iloc[i]/step_up['valuation'].iloc[i-1]*step_up['value_chg'].iloc[i-1]
    else:
        step_up['value_chg'].iloc[i] = 1

100%|██████████████████████████████████████████████████████████████████████████████| 311/311 [00:00<00:00, 1672.04it/s]


In [43]:
step_up['v2/v1'] = ''
step_up['v3/v2'] = ''
step_up['v4/v3'] = ''
step_up.loc[(step_up['rounds']==2),'v2/v1']=step_up['value_chg']
step_up.loc[(step_up['rounds']==3),'v3/v2']=step_up['value_chg']
step_up.loc[(step_up['rounds']==3),'v2/v1']=step_up['v2/v1'].shift()
step_up.loc[(step_up['rounds']==4),'v4/v3']=step_up['value_chg']
step_up.loc[(step_up['rounds']==4),'v3/v2']=step_up['v3/v2'].shift()
step_up.loc[(step_up['rounds']==4),'v2/v1']=step_up['v2/v1'].shift()
step_up = step_up.loc[:,['Company name','rounds','v2/v1','v3/v2','v4/v3']]

In [44]:
step_up.head()

,Company name,rounds,v2/v1,v3/v2,v4/v3
0,ATEEDA,1,,,
1,ATEEDA,2,2.395736,,
2,ATEEDA,3,2.395736,-16.043576,
3,ATEEDA,4,2.395736,-16.043576,1.985387
261,Adbrain,1,,,


In [45]:
df = pd.merge(df,step_up,on=['Company name','rounds'],how='left')
df = df.drop_duplicates(subset=['Company name','last_name','first_name','filing_date'])

In [55]:
df = df.sort_values(by=['Company name','last_name','rounds'])
length_column = df.shape[0]
df['follow_count'] = (0)
for i in tqdm(range(0,length_column)):
    if (df['last_name'].iloc[i] == df['last_name'].iloc[i-1])&(df['first_name'].iloc[i] == df['first_name'].iloc[i-1])&(df['Company name'].iloc[i] == df['Company name'].iloc[i-1]):
        df['follow_count'].iloc[i] = df['follow'].iloc[i]+df['follow_count'].iloc[i-1]

100%|████████████████████████████████████████████████████████████████████████████| 8445/8445 [00:02<00:00, 3324.76it/s]


In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8445 entries, 0 to 9395
Data columns (total 28 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Company name        8445 non-null   object        
 1   filing_date         8445 non-null   datetime64[ns]
 2   total_shares        8445 non-null   float64       
 3   round_shares        8445 non-null   object        
 4   rounds              8445 non-null   object        
 5   first_name          8445 non-null   object        
 6   last_name           8445 non-null   object        
 7   number_of_shares    8445 non-null   float64       
 8   percent_total       8445 non-null   float64       
 9   Deal date           8445 non-null   datetime64[ns]
 10  Investment amount   8445 non-null   float64       
 11  Incorporation date  8241 non-null   object        
 12  Current stage       8445 non-null   object        
 13  Exit date           8445 non-null   datetime64[n

In [58]:
df.to_csv(r'df.csv')  

## Calculate TVPI(total value to paid-in)

In [48]:
df_tvpi = df.sort_values(by=['Company name','last_name','rounds'])
length_column = df_tvpi.shape[0]
df_tvpi['paid_in'] = ''
df_tvpi['follow_count'] = (0)
for i in tqdm(range(0,length_column)):
    if (df_tvpi['last_name'].iloc[i] == df_tvpi['last_name'].iloc[i-1])&(df_tvpi['first_name'].iloc[i] == df_tvpi['first_name'].iloc[i-1])&(df_tvpi['Company name'].iloc[i] == df_tvpi['Company name'].iloc[i-1]):
        df_tvpi['paid_in'].iloc[i] = df_tvpi['valuation'].iloc[i]*df_tvpi['add_shares'].iloc[i]+df_tvpi['paid_in'].iloc[i-1]
        df_tvpi['follow_count'].iloc[i] = df_tvpi['follow'].iloc[i]+df_tvpi['follow_count'].iloc[i-1]
    else:
        df_tvpi['paid_in'].iloc[i] = df_tvpi['valuation'].iloc[i]*df_tvpi['add_shares'].iloc[i]
#calculate the cumulative paid-in of investors

100%|████████████████████████████████████████████████████████████████████████████| 8445/8445 [00:07<00:00, 1094.33it/s]


In [49]:
df_tvpi.loc[df_tvpi['last_name']=='VYZAS']

,Company name,filing_date,total_shares,round_shares,rounds,first_name,last_name,number_of_shares,percent_total,Deal date,...,exit_shares,add_shares,follow,valuation,first_industry,v2/v1,v3/v2,v4/v3,paid_in,follow_count
8749,Upstream,2015-06-13,146053.853296,146053.853296,1,ELIAS,VYZAS,1573.0,1.077,2015-04-30,...,208344.370861,1573.0,1,9.229048,Content and Publishing,,,,14517.29226,0
8755,Upstream,2016-06-12,163005.181347,16951.328051,2,ELIAS,VYZAS,1573.0,0.965,2016-06-01,...,208344.370861,0.0,0,41.907985,Content and Publishing,4.540878,,,14517.29226,0
8761,Upstream,2017-06-12,173428.886439,10423.705092,3,ELIAS,VYZAS,1573.0,0.907,2017-03-09,...,208344.370861,0.0,0,19.183774,Content and Publishing,4.540878,2.07863,,14517.29226,0
8767,Upstream,2018-06-12,208344.370861,34915.484422,4,ELIAS,VYZAS,1573.0,0.755,2017-06-22,...,208344.370861,0.0,0,16.167469,Content and Publishing,4.540878,2.07863,1.751802,14517.29226,0


In [50]:
df_tvpi = df_tvpi.drop_duplicates(subset=['Company name','first_name','last_name'], keep='last')
df_tvpi['total_value'] = df_tvpi['Exit price']/df_tvpi['exit_shares']*df_tvpi['number_of_shares']
df_tvpi['TVPI'] = df_tvpi['total_value']/df_tvpi['paid_in']

In [51]:
df_tvpi.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3613 entries, 3 to 9395
Data columns (total 31 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Company name        3613 non-null   object        
 1   filing_date         3613 non-null   datetime64[ns]
 2   total_shares        3613 non-null   float64       
 3   round_shares        3613 non-null   object        
 4   rounds              3613 non-null   object        
 5   first_name          3613 non-null   object        
 6   last_name           3613 non-null   object        
 7   number_of_shares    3613 non-null   float64       
 8   percent_total       3613 non-null   float64       
 9   Deal date           3613 non-null   datetime64[ns]
 10  Investment amount   3613 non-null   float64       
 11  Incorporation date  3539 non-null   object        
 12  Current stage       3613 non-null   object        
 13  Exit date           3613 non-null   datetime64[n

In [52]:
df_tvpi.to_csv(r'df_tvpi.csv')  

In [53]:
mean_TVPI = np.mean(df_tvpi['TVPI'].values.tolist())
mean_TVPI

0.13248259236984766

In [54]:
# df.columns